In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from keras.applications.vgg16 import preprocess_input

import os
import cv2
from os import walk
import glob as gb
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
SEED = 1000
IMG_SIZE = 224
BATCH_SIZE = 24

TRAIN_DIR = 'C:/Users/Ankur Saha/Thesis/Caltech Bird/Train'
VALID_DIR = 'C:/Users/Ankur Saha/Thesis/Caltech Bird/Validation'
TEST_DIR = 'C:/Users/Ankur Saha/Thesis/Caltech Bird/Test'

In [6]:
from glob import glob
training_dir=TRAIN_DIR
validation_dir=VALID_DIR
image_files = glob(training_dir + '/*/*')
valid_image_files = glob(validation_dir + '/*/*')

In [7]:
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

Total Classes = 199


In [14]:
from keras.models import Model
from keras.layers import Flatten, Dense
from tensorflow.keras.applications.vgg16 import VGG16

#IMG_SIZE=80

IMAGE_SIZE = [IMG_SIZE, IMG_SIZE] 


vgg = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False) 


for layer in vgg.layers:
    layer.trainable = False

x = Flatten()(vgg.output)

x = Dense(num_classes, activation = 'softmax')(x)  

model = Model(inputs = vgg.input, outputs = x)

model.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer=keras.optimizers.Adam(learning_rate=0.0001), 
    metrics=['accuracy']
)

In [15]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [16]:
training_datagen = ImageDataGenerator(
                                    rescale=1./255,
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input)

validation_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)

training_generator = training_datagen.flow_from_directory(
    training_dir, 
    target_size = IMAGE_SIZE, 
    batch_size = BATCH_SIZE, 
    class_mode = 'sparse')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    target_size = IMAGE_SIZE, 
    batch_size = BATCH_SIZE, 
    class_mode = 'sparse')

Found 8190 images belonging to 199 classes.
Found 2311 images belonging to 199 classes.


In [17]:
history = model.fit(training_generator,
                   steps_per_epoch = 20,
                   epochs = 50,
                   validation_data = validation_generator,
                   )

Epoch 1/50
20/20 [==============================] - 11s 558ms/step - loss: 5.9015 - accuracy: 0.0043 - val_loss: 5.8930 - val_accuracy: 0.0108
Epoch 2/50
20/20 [==============================] - 10s 499ms/step - loss: 5.6531 - accuracy: 0.0292 - val_loss: 5.3655 - val_accuracy: 0.0199
Epoch 3/50
20/20 [==============================] - 10s 507ms/step - loss: 5.1088 - accuracy: 0.0562 - val_loss: 5.1427 - val_accuracy: 0.0381
Epoch 4/50
20/20 [==============================] - 10s 507ms/step - loss: 4.9424 - accuracy: 0.0750 - val_loss: 5.0377 - val_accuracy: 0.0394
Epoch 5/50
20/20 [==============================] - 10s 507ms/step - loss: 4.8735 - accuracy: 0.0708 - val_loss: 4.8831 - val_accuracy: 0.0459
Epoch 6/50
20/20 [==============================] - 10s 510ms/step - loss: 4.6610 - accuracy: 0.0979 - val_loss: 4.7508 - val_accuracy: 0.0619
Epoch 7/50
20/20 [==============================] - 10s 508ms/step - loss: 4.4938 - accuracy: 0.1167 - val_loss: 4.7343 - val_accuracy: 0.0550